

*   mean rating of each image
*   spearman's rank correlation - reliability: Across 25 random splits, the average reliability across all eight categories was ..., min/max category.
*   compute correlation with models (pearson rank correlations?)
*   find "outliers" to compare scores, e.g. bananas with spots, image of groups of items 
*   the role of feature complexity

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats

## Data reliability ##

In [6]:
path='/content/drive/MyDrive/1016/Project/Typicality_project/Analysis/human_rating_bycat'
# if windows
# path = 'human_rating_bycat'

banana=pd.read_csv(path+'/banana.csv')
beach=pd.read_csv(path+'/beach.csv')
car=pd.read_csv(path+'/car.csv')
church=pd.read_csv(path+'/church.csv')
lighthouse=pd.read_csv(path+'/lighthouse.csv')
mountain=pd.read_csv(path+'/mountain.csv')
mug=pd.read_csv(path+'/mug.csv')
plane=pd.read_csv(path+'/plane.csv')

In [ ]:
df_lst = [banana, beach, car, church, lighthouse, mountain,
       mug, plane]
df_name = ['Banana', 'Beach', 'Car', 'Church', 'Lighthouse', 'Mountain',
       'Mug', 'Plane']

for idx, df in enumerate(df_lst):
    plt.hist(df.mean())
    plt.title(df_name[idx])
    plt.show()

In [8]:
def compute_reliability(data):
  reliability=[]
  for i in range(25):
    b1, b2=train_test_split(data, test_size=0.5)
    b1, b2=b1.mean(), b2.mean()
    coef, p=spearmanr(b1, b2)
    reliability.append(coef)
  return np.mean(reliability)

In [9]:
res={}
data=[banana, beach, car, church, lighthouse, mountain, mug, plane]
for i, cat in enumerate(data):
  res[i]=compute_reliability(cat)

In [10]:
res

{0: 0.8827992334314991,
 1: 0.961445396533741,
 2: 0.945821944554654,
 3: 0.8987446421656933,
 4: 0.906025131621308,
 5: 0.9152394833383266,
 6: 0.8644567316553384,
 7: 0.9485545796994741}

In [11]:
print('min reliability: ', min(res.values()))
print('max reliability: ', max(res.values()))
print('mean reliability: ', sum(res.values())/8)

min reliability:  0.8644567316553384
max reliability:  0.961445396533741
mean reliability:  0.9153858928750043


In [12]:
'''
print('banana reliability: ', compute_reliability(banana))
print('beach reliability: ', compute_reliability(beach))
print('car reliability: ', compute_reliability(car))
print('church reliability: ', compute_reliability(church))
print('lighthouse reliability: ', compute_reliability(lighthouse))
print('mountain reliability: ', compute_reliability(mountain))
print('mug reliability: ', compute_reliability(mug))
print('plane reliability: ', compute_reliability(plane))
'''

"\nprint('banana reliability: ', compute_reliability(banana))\nprint('beach reliability: ', compute_reliability(beach))\nprint('car reliability: ', compute_reliability(car))\nprint('church reliability: ', compute_reliability(church))\nprint('lighthouse reliability: ', compute_reliability(lighthouse))\nprint('mountain reliability: ', compute_reliability(mountain))\nprint('mug reliability: ', compute_reliability(mug))\nprint('plane reliability: ', compute_reliability(plane))\n"

## Correlation to models ##

Problem - different order of images in survey and model?

In [ ]:
# If windows
# google_df=pd.read_csv('../GoogleNet/googlenet_scores_wordnet_id_closest_label.csv')
# vgg_df=pd.read_csv('../VGG/vgg_scores_wordnet_id_closest_label.csv')
# alex_df=pd.read_csv('../alexnet/alexnet_scores_wordnet_id_closest_label.csv')
overfeat_df = pd.read_csv('/content/drive/MyDrive/1016/Project/Typicality_project/Overfeat/overfeat_scores_wordnet_id_closest_label.csv',index_col=0)
overfeat_df.head()

In [50]:
overfeat_df['dir'] = overfeat_df['dir'].astype('int')

In [52]:
overfeat_df = overfeat_df.sort_values(by=['type','dir'])
overfeat_df

,type,dir,id_labels,readable_labels,similarity_score,closest_id_label,closest_similarity_score,closest_readable_label,closest_prob
112,Banana,1,"{'07753592-n': 0.980348, '07717410-n': 0.00572...","{'banana': 0.980348, 'acorn squash': 0.0057260...","{'07753592-n': 1.0, '07717410-n': 0.1428571428...",07753592-n,1.0,banana,0.980348
120,Banana,2,"{'02606052-n': 0.41746099999999997, '07753592-...","{'rock beauty, Holocanthus tricolor': 0.417460...","{'02606052-n': 0.047619047619047616, '07753592...",07753592-n,1.0,banana,0.123135
121,Banana,3,"{'07753592-n': 0.41874700000000004, '03065424-...","{'banana': 0.41874700000000004, 'coil, spiral,...","{'07753592-n': 1.0, '03065424-n': 0.0909090909...",07753592-n,1.0,banana,0.418747
122,Banana,4,"{'07753592-n': 0.618004, '07720875-n': 0.14292...","{'banana': 0.618004, 'bell pepper': 0.142925, ...","{'07753592-n': 1.0, '07720875-n': 0.125, '0771...",07753592-n,1.0,banana,0.618004
123,Banana,5,"{'07753592-n': 0.364579, '07930864-n': 0.12127...","{'banana': 0.364579, 'cup': 0.121272, 'coil, s...","{'07753592-n': 1.0, '07930864-n': 0.0769230769...",07753592-n,1.0,banana,0.364579
...,...,...,...,...,...,...,...,...,...
3,Plane,12,"{'04592741-n': 0.318848, '03874293-n': 0.16218...","{'wing': 0.318848, 'paddlewheel, paddle wheel'...","{'04592741-n': 0.1, '03874293-n': 0.0769230769...",02690373-n,0.5,airliner,0.113196
4,Plane,13,"{'04552348-n': 0.145282, '04065272-n': 0.14077...","{'warplane, military plane': 0.145282, 'recrea...","{'04552348-n': 0.3333333333333333, '04065272-n...",02690373-n,0.5,airliner,0.057365
5,Plane,14,"{'03933933-n': 0.352139, '03160309-n': 0.29153...","{'pier': 0.352139, 'dam, dike, dyke': 0.291537...","{'03933933-n': 0.1, '03160309-n': 0.0833333333...",03933933-n,0.1,pier,0.352139
6,Plane,15,"{'04273569-n': 0.9261280000000001, '04483307-n...","{'speedboat': 0.9261280000000001, 'trimaran': ...","{'04273569-n': 0.125, '04483307-n': 0.125, '02...",02690373-n,0.5,airliner,0.001270


In [ ]:
# with GoogLeNet
google_df=pd.read_csv('../content/drive/MyDrive/1016/Project/Typicality_project/GoogleNet/googlenet_scores_wordnet_id_closest_label.csv')
vgg_df=pd.read_csv('../content/drive/MyDrive/1016/Project/Typicality_project/VGG/vgg_scores_wordnet_id_closest_label.csv')
alex_df=pd.read_csv('../content/drive/MyDrive/1016/Project/Typicality_project/alexnet/alexnet_scores_wordnet_id_closest_label.csv')

google_df.head()

In [54]:
Bana=[google_df[google_df['type']=='Banana']['closest_prob'], 
      alex_df[alex_df['type']=='Banana']['closest_prob'],
      vgg_df[vgg_df['type']=='Banana']['closest_prob'],
      overfeat_df[overfeat_df['type']=='Banana']['closest_prob']]

Beach=[google_df[google_df['type']=='Beach']['closest_prob'],
       alex_df[alex_df['type']=='Beach']['closest_prob'],
       vgg_df[vgg_df['type']=='Beach']['closest_prob'],
       overfeat_df[overfeat_df['type']=='Beach']['closest_prob']]

Car=[google_df[google_df['type']=='Car']['closest_prob'],
     alex_df[alex_df['type']=='Car']['closest_prob'],
     vgg_df[vgg_df['type']=='Car']['closest_prob'],
     overfeat_df[overfeat_df['type']=='Car']['closest_prob']]

Church=[google_df[google_df['type']=='Church']['closest_prob'],
        alex_df[alex_df['type']=='Church']['closest_prob'],
        vgg_df[vgg_df['type']=='Church']['closest_prob'],
        overfeat_df[overfeat_df['type']=='Church']['closest_prob']]

Light=[google_df[google_df['type']=='Lighthouse']['closest_prob'],
       alex_df[alex_df['type']=='Lighthouse']['closest_prob'],
       vgg_df[vgg_df['type']=='Lighthouse']['closest_prob'],
       overfeat_df[overfeat_df['type']=='Lighthouse']['closest_prob']]

Mount=[google_df[google_df['type']=='Mountain']['closest_prob'],
       alex_df[alex_df['type']=='Mountain']['closest_prob'],
       vgg_df[vgg_df['type']=='Mountain']['closest_prob'],
       overfeat_df[overfeat_df['type']=='Mountain']['closest_prob']]

Mug=[google_df[google_df['type']=='Mug']['closest_prob'],
     alex_df[alex_df['type']=='Mug']['closest_prob'],
     vgg_df[vgg_df['type']=='Mug']['closest_prob'],
     overfeat_df[overfeat_df['type']=='Mug']['closest_prob']]

Plane=[google_df[google_df['type']=='Plane']['closest_prob'],
       alex_df[alex_df['type']=='Plane']['closest_prob'],
       vgg_df[vgg_df['type']=='Plane']['closest_prob'],
       overfeat_df[overfeat_df['type']=='Plane']['closest_prob']]

# Overfeat

In [55]:
#bana=banana.mean()
#Bana=Bana.reset_index(drop=True, inplace=True)
corr=[]
for i in range(banana.shape[0]):
  if i!=17 and i!=21:
    bana=banana.iloc[i]
    coef, p=stats.pearsonr(bana.values, Bana[3].values)
    print(coef)
    #coef=np.corrcoef(bana, Bana)
    #coef, p=spearmanr(bana, Bana)
    corr.append(abs(coef))
bana_corr=np.mean(corr)
print('Banana correlation: ', bana_corr)

0.7525478449133675
0.38217886941039725
0.6156713031141431
0.4872118499957767
0.5447223469535333
0.4602924941427539
0.3736638637488939
0.22783782646499803
0.5504527529032759
0.45499724666253705
0.6792142910954797
0.13193140096621217
0.6213501594965359
0.444960663449721
0.6771121174615106
0.4287188490187555
0.4953798315938281
0.6243684056945796
0.4709743550340037
0.633064446824263
0.4687072148569925
0.40727334714998104
0.40727334714998104
0.48785796196734454
0.2835042885050709
0.6264488933530588
0.40727334714998104
0.702268714461332
0.46409858721361946
0.40727334714998104
0.5160746160130258
0.5484026793006227
0.18403913186483456
0.5268538314728851
Banana correlation:  0.48511765372215515


In [56]:
corr=[]
for i in range(beach.shape[0]):
  #if i!=17 and i!=21:
  bea=beach.iloc[i]
  coef, p=stats.pearsonr(bea.values, Beach[3].values)
  print(coef)
  #coef=np.corrcoef(bea, Beach)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
bea_corr=np.mean(corr)
print('Beach correlation: ', bea_corr)

0.1397138594301332
0.22812061349040513
-0.27640833937019627
0.032492787070559276
0.269391584621871
0.09036420079798217
0.08787324234141773
0.37667540529100946
-0.19714423164400616
-0.3075636997955658
0.12106011426129065
-0.2139881901367406
-0.1692510357964329
-0.04588763125529918
-0.243986758224398
-0.07412710711533402
-0.02625487064351159
-0.16330665982462492
0.05978552836647992
-0.007187953901836727
0.40534163952315216
-0.2568150044676176
-0.1773060850354131
0.05514014605033768
-0.0932783448782233
0.01951190860751846
0.04386752189449066
-0.04598933814363167
0.010077030375622534
0.24633620418955884
0.04277206525235262
-0.06955057871592757
0.1892630153975027
0.2949642416366595
0.3555760827473881
-0.05724307614298329
Beach correlation:  0.017862174618166363


In [57]:
corr=[]
for i in range(car.shape[0]):
  #if i!=17 and i!=21:
  ca=car.iloc[i]
  coef, p=stats.pearsonr(ca.values, Car[3].sort_index().values)
  print(coef)
  #coef=np.corrcoef(ca, Car)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
car_corr=np.mean(corr)
print('Car correlation: ', car_corr)

-0.5585590908322242
-0.5780887792086912
-0.47615164417329764
-0.4598211777249163
-0.512274396608509
-0.3114767577712535
-0.04458078912054647
-0.5704231183756417
0.004814982364936299
-0.19888883479986594
-0.32146161059482603
-0.39311319318421944
-0.42847193404952905
-0.2997015734375358
-0.5650132567598767
-0.591096979702443
-0.5572882117827046
-0.4643844729319757
-0.5823510230791871
-0.5588717019123115
-0.4746105487446607
-0.2551994865018357
-0.3618477868547648
-0.46192846790510267
-0.42331407667343596
-0.34577470380142383
-0.34614808766327976
-0.2450399203982684
-0.39237623384297776
-0.6370199033441362
-0.4083888842113849
-0.3711043723193542
-0.6885852603384874
-0.5024369547218268
-0.33616896364001486
-0.11865349143404415
Car correlation:  -0.41210557516887825


In [58]:
corr=[]
for i in range(church.shape[0]):
  #if i!=17 and i!=21:
  ch=church.iloc[i]
  coef, p=stats.pearsonr(ch.values, Church[3].values)
  print(coef)
  #coef=np.corrcoef(ch, Church)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
church_corr=np.mean(corr)
print('Church correlation: ', church_corr)

0.29211980595063564
0.21146885450424446
0.34647641718309097
0.11756862369723614
0.11821367514809494
0.22645978108201864
-0.25801014316385074
0.2033173296345609
0.10910486503138991
0.15568549538815274
0.5453970575782292
0.12209917426834695
0.29922304242708986
0.4553343958958551
0.4637364270305442
0.057250081567086636
-0.007338665837687525
-0.18501533397532582
0.6759396717399389
0.32641600581020014
0.3988980193915289
0.26384770943529906
-0.04262183150453878
-0.04028847024720622
0.07991503979357327
0.32392363112625705
0.5141461841867154
-0.2141383018360884
-0.024729742594791215
0.07867873221420124
0.3609795274007464
0.40825384601905196
0.6266863145419246
0.022513259198047197
0.23516793954124812
0.1632049085647573
Church correlation:  0.20638564794973824


In [59]:
corr=[]
for i in range(lighthouse.shape[0]):
  #if i!=17 and i!=21:
  li=lighthouse.iloc[i]
  coef, p=stats.pearsonr(li.values, Light[3].values)
  print(coef)
  #coef=np.corrcoef(li, Light)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
light_corr=np.mean(corr)
print('Lighthouse correlation: ', light_corr)

0.5550087939521223
0.5190539292016654
0.5945604020905829
0.43975477284680536
0.6241588436116132
0.35579196666553503
0.42149884989814235
0.32048403782867757
0.2716569533451296
0.5224307852548764
0.4400407878860087
0.40209420075263735
0.5313931046854139
0.6447010259088555
0.6552303844779734
0.30526680223818997
0.13850251692666127
0.3655756125016404
0.2615139693642133
0.3383711314003172
0.3290760497814422
0.3144280618066335
0.35047450618334586
0.22942231396732354
0.6740400312602886
0.49318753981657026
0.3920646798133487
0.2450471834939125
0.4976664162043229
0.507240832642557
0.4662833198513847
0.30444568156299834
0.5307809442984353
0.45924026004941726
0.5221363459444124
0.48882942100593463
Lighthouse correlation:  0.4308736794033164


In [60]:
corr=[]
for i in range(mountain.shape[0]):
  if i!=31:
    mount=mountain.iloc[i]
    coef, p=stats.pearsonr(mount.values, Mount[3].values)
    print(coef)
  #coef=np.corrcoef(mount, Mount)
    #coef, p=spearmanr(bana, Bana)
    corr.append(coef)
mount_corr=np.mean(corr)
print('Mountain correlation: ', mount_corr)

0.49400187800170564
0.4041819800811123
0.624542170312138
0.27251040498390555
0.31115758464692017
0.35204997434692653
0.4601804189194829
0.1668812463836925
0.3255872307288502
0.35177333888094375
0.11049535740528053
0.32509711776389527
-0.010121305416835341
0.5038335666201246
0.4336418267025898
0.27746934464862144
0.3424377558210887
0.3429201691304182
0.37664798161178864
0.42907366266033
0.30941948525969865
-0.010121305416835348
-0.010121305416835348
0.45692833332631805
-0.010121305416835341
0.2204172691571345
0.04306363565495496
0.5049564500928551
0.18661693270778826
0.36511281092520903
0.5592206536306439
0.444118612560257
0.37571552456770146
0.2955186564644831
0.501947340123859
Mountain correlation:  0.317915242641525


In [61]:
corr=[]
for i in range(mug.shape[0]):
  #if i!=17 and i!=21:
  m=mug.iloc[i]
  coef, p=stats.pearsonr(m.values, Mug[3].values)
  print(coef)
  #coef=np.corrcoef(m, Mug)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
mug_corr=np.mean(corr)
print('Mug correlation: ', mug_corr)

0.019363121149863577
0.2821209406246138
0.26331084781791203
0.12971416847811337
-0.14262814862554696
0.07047704458027357
-0.13620411573537447
0.21205750186161815
0.14492293754176439
-0.14774070023343372
-0.20373435634395842
0.30796613054031136
0.19619954268828707
0.2380097219976827
0.05755969542648298
0.03408908406600987
0.2325903924608069
0.31467981957650826
-0.22306801204158727
0.0005194517298946974
0.12402222985567934
0.14178144480051263
0.04127509453801836
0.07791525037803587
0.32331909618510407
-0.03791221103003356
0.17963043580767485
0.03988275837330856
-0.05594632083855228
-0.04443042727236267
0.1892293496014408
0.1011062413037928
-0.25748413101753714
0.39341149447804147
-0.04077680391835853
-0.1617367112686045
Mug correlation:  0.07398588493156671


In [62]:
corr=[]
for i in range(plane.shape[0]):
  #if i!=17 and i!=21:
  pla=plane.iloc[i]
  coef, p=stats.pearsonr(pla.values, Plane[3].values)
  print(coef)
  #coef=np.corrcoef(pla, Plane)
    #coef, p=spearmanr(bana, Bana)
  corr.append(coef)
plane_corr=np.mean(corr)
print('Plane correlation: ', plane_corr)

0.3733783138972454
0.328825411045726
0.5263725808814399
0.5526736805870113
0.339722562328166
0.3886173535172511
0.5618211014482211
0.30382490110754745
0.2182592344565666
0.3197542396195615
0.7597886915137337
0.29972537043928504
0.4495305498810829
0.38084089429788653
0.4893372283153249
0.43390265390310306
0.4520875308337081
0.6808553958937591
0.16992704661090474
0.5161849191437232
0.22037173574497482
0.2139274794057392
0.16600294751536135
0.4248239741606915
0.41680710525062103
0.2873750691653437
0.4353962414815016
0.04347484823300499
0.35895768935742367
0.39853888043577435
0.4776110346522841
0.22821228664045615
0.39057783755049325
0.5877448319962606
0.5130355145793004
0.5049356864335415
Plane correlation:  0.3948117450645561


In [63]:
sum_corr=bana_corr+bea_corr+car_corr+church_corr+mount_corr+mug_corr+plane_corr+light_corr
avg_corr=sum_corr/8
avg_corr

0.1893558066452682